In [1]:
print('tes123')

tes123


In [ ]:
# Path: main.ipynb
print('tes123')

In [10]:
import pandas as pd

dataset = pd.read_excel('Dataset.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'Dataset.xlsx'